In [18]:

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [19]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
# documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()


In [20]:
from llama_index.core import VectorStoreIndex

# Create an index from the documents
index = VectorStoreIndex.from_documents(documents=documents)

# Query engine for handling queries
query_engine = index.as_query_engine()


In [21]:
from llama_index.core.workflow import Event
from typing import Dict, List, Any
from llama_index.core.schema import NodeWithScore


class QueryMultiStepEvent(Event):
    """
    Event containing results of a multi-step query process.

    Attributes:
        nodes (List[NodeWithScore]): Nodes with scores.
        source_nodes (List[NodeWithScore]): Source nodes.
        final_response_metadata (Dict[str, Any]): Metadata for the response.
    """
    nodes: List[NodeWithScore]
    source_nodes: List[NodeWithScore]
    final_response_metadata: Dict[str, Any]


In [25]:
from llama_index.core.workflow import (
    Workflow, Context, StartEvent, StopEvent, step
)
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)
from llama_index.core.schema import QueryBundle, TextNode
from typing import cast, Dict, Any


class MultiStepQueryEngineWorkflow(Workflow):

    def combine_queries(
        self, query_bundle: QueryBundle, prev_reasoning: str, index_summary: str, llm
    ) -> QueryBundle:
        """
        Dynamically refine the query for the next step using LLM.

        Parameters:
        - query_bundle: Current query.
        - prev_reasoning: All prior questions and answers.
        - index_summary: High-level summary of the index data.
        """
        prompt_template = (
            "You are solving a multi-step question step by step.\n"
            "Index Summary: {index_summary}\n\n"
            "Previous Steps and Answers:\n{prev_reasoning}\n\n"
            "Current Query: {current_query}\n\n"
            "Based on the above context, generate the next refined sub-question. "
            "If no further refinement is needed, respond with 'None'.\n\n"
            "Refined Query:"
        )

        # Build prompt with current context
        detailed_prompt = prompt_template.format(
            index_summary=index_summary,
            prev_reasoning=prev_reasoning.strip(),
            current_query=query_bundle.query_str,
        )

        # Generate the next refined query using the LLM
        refined_query = llm.complete(detailed_prompt).text.strip()

        # Handle stopping condition
        if refined_query.lower() in ["none", "no query", "stop"]:
            refined_query = "None"

        return QueryBundle(query_str=refined_query)



    def default_stop_fn(self, stop_dict: Dict) -> bool:
        query_bundle = cast(QueryBundle, stop_dict.get("query_bundle"))
        return "none" in query_bundle.query_str.lower()

    @step(pass_context=True)
    async def query_multistep(self, ctx: Context, ev: StartEvent) -> QueryMultiStepEvent:
        """Execute the multi-step query process."""
        prev_reasoning, cur_steps = "", 0
        should_stop = False

        # Extract required inputs
        num_steps = ev.get("num_steps", 3)
        query = ev.get("query")
        if not query:
            raise ValueError("Query must be provided in StartEvent.")
        
        index_summary = ev.get("index_summary", "")
        llm, query_engine = Settings.llm, ev.get("query_engine")

        # Initialize metadata and results
        text_chunks, source_nodes = [], []
        final_response_metadata: Dict[str, Any] = {"sub_qa": []}

        # Set initial query into the context
        await ctx.set("query", query)

        while not should_stop:
            if cur_steps >= num_steps:
                break

            # Retrieve the query dynamically from context
            query = await ctx.get("query")

            # Generate refined query
            updated_query_bundle = self.combine_queries(
                QueryBundle(query_str=query), prev_reasoning, index_summary, llm
            )

            # Check for stop condition
            if "none" in updated_query_bundle.query_str.lower():
                should_stop = True
                break

            print(f"Step {cur_steps}: {updated_query_bundle.query_str}")

            # Execute the query and append results
            cur_response = query_engine.query(updated_query_bundle)

            # Update context and reasoning
            await ctx.set("query", updated_query_bundle.query_str)  # Set refined query back in context
            cur_qa_text = f"Q: {updated_query_bundle.query_str}\nA: {cur_response}"
            text_chunks.append(cur_qa_text)
            source_nodes.extend(cur_response.source_nodes)

            # Update metadata
            final_response_metadata["sub_qa"].append((updated_query_bundle.query_str, cur_response))
            prev_reasoning += f"- Question: {updated_query_bundle.query_str}\n- Answer: {cur_response}\n"
            cur_steps += 1

        # Prepare response event
        nodes = [NodeWithScore(node=TextNode(text=text_chunk)) for text_chunk in text_chunks]
        return QueryMultiStepEvent(nodes=nodes, source_nodes=source_nodes, final_response_metadata=final_response_metadata)


    @step(pass_context=True)
    async def synthesize(self, ctx: Context, ev: QueryMultiStepEvent) -> StopEvent:
        response_synthesizer = get_response_synthesizer()
        query = await ctx.get("query")
        final_response = await response_synthesizer.asynthesize(
            query=query, nodes=ev.nodes, additional_source_nodes=ev.source_nodes
        )
        final_response.metadata = ev.final_response_metadata
        return StopEvent(result=final_response)


In [26]:
# Initialize the workflow
w = MultiStepQueryEngineWorkflow(timeout=200)

async def main():
    result = await w.run(
        query="In which city did the author found his first company, Viaweb?",
        query_engine=query_engine,
        index_summary="Used to answer questions about the author and Viaweb.",
        num_steps=3,
    )
    print("\nFinal Result:")
    print(result)
    print("\nStep-by-Step Reasoning:")
    sub_qa = result.metadata["sub_qa"]
    for idx, (question, answer) in enumerate(sub_qa):
        print(f"Step {idx}:")
        print(f"  Question: {question}")
        print(f"  Answer: {answer}")



In [ ]:
await main()

In [ ]:
from llama_index.core.workflow import draw_all_possible_flows

draw_all_possible_flows(w, filename="workflow_capstone.html")